In [1]:
import formulas
import storage
import pandas as pd
import os
import re



if __name__ == '__main__':

    website_creator_path = os.getcwd()
    formula_file_path = website_creator_path + os.path.sep + 'formulas.csv'
    order_file_path = website_creator_path + os.path.sep + 'sort_orders.csv'
    docs_dir = storage.get_docs_path(website_creator_path)
    
    df_formulas_by_year =  storage.get_formulas_by_year_df(formula_file_path)
    sort_orders = pd.read_csv(order_file_path)
    
    storage.delete_directory_if_it_exists(docs_dir)

    # Create formula by year summary
    formula_by_year_summary_df = formulas.get_formula_by_year_summary_df(df_formulas_by_year)
    storage.create_sub_directories(docs_dir, formula_by_year_summary_df)
    regex_id_for_formula_dirs = '.*' + os.path.sep + 'Formulas*'
    storage.create_index_files(base_dir=docs_dir, folder_regex=regex_id_for_formula_dirs, 
                               book_collapse=True, df_sort_orders=sort_orders)
    # # # formulas.create_formula_files(docs_dir, df_formulas)

    # df_calculus = formulas.df_calculus_summary(df_formulas)
    # styler_calculus = formulas.styler_calculus_summary(df_calculus, formulas.formulas_on_formula_sheet(df_formulas))
    # display(styler_calculus)